In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# 1. 데이터 로드
data = pd.read_csv("../../../mon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label']

# 2. 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# 3. 데이터 나누기: 훈련, 검증, 테스트 데이터셋으로 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)  # 테스트 30%, 나머지 훈련+검증 70%

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)  # 검증 20% (전체의 14%)

# 4. 파이프라인 설정
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# 5. 하이퍼파라미터 그리드 정의
param_grid = {
    'classifier__criterion': ['gini', 'entropy'],       # 분할 기준
    'classifier__max_depth': [10, 15, 20],             # 트리의 최대 깊이
    'classifier__min_samples_split': [2, 5, 10],       # 노드 분할 최소 샘플 수
    'classifier__min_samples_leaf': [1, 2, 4]          # 리프 노드의 최소 샘플 수
}

# 6. 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 7. GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# 8. 모델 학습 (최적의 하이퍼파라미터 탐색)
grid_search.fit(X_train, y_train)

# 9. 최적의 하이퍼파라미터 확인
best_model = grid_search.best_estimator_
print("최적의 하이퍼파라미터: ", grid_search.best_params_)

# 검증 데이터 평가
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred, average='macro')  # 다중분류용 F1 Score
print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score (Macro): {val_f1:.2f}")

# 10. 테스트 데이터 평가
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average='macro')  # 다중분류용 F1 Score
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1 Score (Macro): {test_f1:.4f}")

# 다중분류용 Precision, Recall 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')

print(f"Test Precision (Macro): {precision:.4f}")
print(f"Test Recall (Macro): {recall:.4f}")

# 11. 분류 보고서 출력
print("\n테스트 데이터에 대한 분류 보고서:")
print(classification_report(y_test, y_test_pred))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
최적의 하이퍼파라미터:  {'classifier__criterion': 'entropy', 'classifier__max_depth': 20, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
Validation Accuracy: 0.65
Validation F1 Score (Macro): 0.65
Test Accuracy: 0.6435
Test F1 Score (Macro): 0.6430
Test Precision (Macro): 0.6476
Test Recall (Macro): 0.6435

테스트 데이터에 대한 분류 보고서:
              precision    recall  f1-score   support

           0       0.51      0.52      0.51        60
           1       0.56      0.53      0.55        60
           2       0.87      0.78      0.82        60
           3       0.78      0.77      0.77        60
           4       0.61      0.62      0.61        60
           5       0.71      0.57      0.63        60
           6       0.71      0.77      0.74        60
           7       0.63      0.62      0.62        60
           8       0.72      0.70      0.71        60
           9       0.67      0.72      0.69        60
